# Advanced Topic: Using External C++ Functions

This is based on the relevant portion of the CmdStan documentation [here](https://mc-stan.org/docs/cmdstan-guide/using-external-cpp-code.html)

Consider the following Stan model, based on the bernoulli example.

In [1]:
import os
try:
    os.remove('bernoulli_external')
except:
    pass

In [2]:
from cmdstanpy import CmdStanModel
model_external = CmdStanModel(stan_file='bernoulli_external.stan', compile=False)
print(model_external.code())

/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


As you can see, it features a function declaration for `make_odds`, but no definition. If we try to compile this, we will get an error. 

In [3]:
model_external.compile()

20:25:18 - cmdstanpy - INFO - compiling stan file /home/runner/work/cmdstanpy/cmdstanpy/docsrc/examples/bernoulli_external.stan to exe file /home/runner/work/cmdstanpy/cmdstanpy/docsrc/examples/bernoulli_external


20:25:24 - cmdstanpy - INFO - compiled model executable: /home/runner/work/cmdstanpy/cmdstanpy/docsrc/examples/bernoulli_external


20:25:24 - cmdstanpy - WARNING - Stan compiler has produced 1 warnings:


20:25:24 - cmdstanpy - WARNING - make[2]: Entering directory '/home/runner/.cmdstan/cmdstan-2.29.2'

--- Translating Stan model to C++ code ---
bin/stanc  --o=/home/runner/work/cmdstanpy/cmdstanpy/docsrc/examples/bernoulli_external.hpp /home/runner/work/cmdstanpy/cmdstanpy/docsrc/examples/bernoulli_external.stan
    '/home/runner/work/cmdstanpy/cmdstanpy/docsrc/examples/bernoulli_external.stan'
    detected; this is a valid stan model but likely unintended!

--- Compiling, linking C++ code ---
g++ -std=c++1y -pthread -D_REENTRANT -Wno-sign-compare -Wno-ignored-attributes      -I stan/lib/stan_math/lib/tbb_2020.3/include    -O3 -I src -I stan/src -I lib/rapidjson_1.1.0/ -I lib/CLI11-1.9.1/ -I stan/lib/stan_math/ -I stan/lib/stan_math/lib/eigen_3.3.9 -I stan/lib/stan_math/lib/boost_1.75.0 -I stan/lib/stan_math/lib/sundials_6.0.0/include -I stan/lib/stan_math/lib/sundials_6.0.0/src/sundials    -DBOOST_DISABLE_ASSERTS          -c -Wno-ignored-attributes   -x c++ -o /home/runner/work/cmdsta

Even enabling the `--allow-undefined` flag to stanc3 will not allow this model to be compiled quite yet.

In [4]:
model_external.compile(stanc_options={'allow-undefined':True})

20:25:24 - cmdstanpy - INFO - compiling stan file /home/runner/work/cmdstanpy/cmdstanpy/docsrc/examples/bernoulli_external.stan to exe file /home/runner/work/cmdstanpy/cmdstanpy/docsrc/examples/bernoulli_external


20:25:24 - cmdstanpy - WARNING - Stan compiler has produced 1 warnings:


20:25:24 - cmdstanpy - WARNING - make[2]: Entering directory '/home/runner/.cmdstan/cmdstan-2.29.2'

--- Translating Stan model to C++ code ---
bin/stanc --allow-undefined --o=/home/runner/work/cmdstanpy/cmdstanpy/docsrc/examples/bernoulli_external.hpp /home/runner/work/cmdstanpy/cmdstanpy/docsrc/examples/bernoulli_external.stan
    '/home/runner/work/cmdstanpy/cmdstanpy/docsrc/examples/bernoulli_external.stan'
    detected; this is a valid stan model but likely unintended!

--- Compiling, linking C++ code ---
g++ -std=c++1y -pthread -D_REENTRANT -Wno-sign-compare -Wno-ignored-attributes      -I stan/lib/stan_math/lib/tbb_2020.3/include    -O3 -I src -I stan/src -I lib/rapidjson_1.1.0/ -I lib/CLI11-1.9.1/ -I stan/lib/stan_math/ -I stan/lib/stan_math/lib/eigen_3.3.9 -I stan/lib/stan_math/lib/boost_1.75.0 -I stan/lib/stan_math/lib/sundials_6.0.0/include -I stan/lib/stan_math/lib/sundials_6.0.0/src/sundials    -DBOOST_DISABLE_ASSERTS          -c -Wno-ignored-attributes   -include /home/ru

To resolve this, we need to both tell the Stan compiler an undefined function is okay **and** let C++ know what it should be. 

We can provide a definition in a C++ header file by using the `user_header` argument to either the CmdStanModel constructor or the `compile` method. 

This will enables the `allow-undefined` flag automatically.

In [5]:
model_external.compile(user_header='make_odds.hpp')

20:25:24 - cmdstanpy - INFO - compiling stan file /home/runner/work/cmdstanpy/cmdstanpy/docsrc/examples/bernoulli_external.stan to exe file /home/runner/work/cmdstanpy/cmdstanpy/docsrc/examples/bernoulli_external


20:25:39 - cmdstanpy - INFO - compiled model executable: /home/runner/work/cmdstanpy/cmdstanpy/docsrc/examples/bernoulli_external


20:25:39 - cmdstanpy - WARNING - Stan compiler has produced 1 warnings:


20:25:39 - cmdstanpy - WARNING - make[2]: Entering directory '/home/runner/.cmdstan/cmdstan-2.29.2'

--- Translating Stan model to C++ code ---
bin/stanc --allow-undefined --o=/home/runner/work/cmdstanpy/cmdstanpy/docsrc/examples/bernoulli_external.hpp /home/runner/work/cmdstanpy/cmdstanpy/docsrc/examples/bernoulli_external.stan
    '/home/runner/work/cmdstanpy/cmdstanpy/docsrc/examples/bernoulli_external.stan'
    detected; this is a valid stan model but likely unintended!

--- Compiling, linking C++ code ---
g++ -std=c++1y -pthread -D_REENTRANT -Wno-sign-compare -Wno-ignored-attributes      -I stan/lib/stan_math/lib/tbb_2020.3/include    -O3 -I src -I stan/src -I lib/rapidjson_1.1.0/ -I lib/CLI11-1.9.1/ -I stan/lib/stan_math/ -I stan/lib/stan_math/lib/eigen_3.3.9 -I stan/lib/stan_math/lib/boost_1.75.0 -I stan/lib/stan_math/lib/sundials_6.0.0/include -I stan/lib/stan_math/lib/sundials_6.0.0/src/sundials    -DBOOST_DISABLE_ASSERTS          -c -Wno-ignored-attributes   -include /home/ru

We can then run this model and inspect the output

In [6]:
fit = model_external.sample(data={'N':10, 'y':[0,1,0,0,0,0,0,0,0,1]})
fit.stan_variable('odds')

20:25:39 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

...


chain 2 |██████████| 00:00 Sampling completed


chain 3 |██████████| 00:00 Sampling completed


chain 4 |██████████| 00:00 Sampling completed


20:25:39 - cmdstanpy - INFO - CmdStan done processing.


ValueError: Unknown variable name: odds
Available variables are 

The contents of this header file are a bit complicated unless you are familiar with the C++ internals of Stan, so they are presented without comment:

```c++
#include <boost/math/tools/promotion.hpp>
#include <ostream>

namespace bernoulli_model_namespace {
    template <typename T0__>  inline  typename
          boost::math::tools::promote_args<T0__>::type  
          make_odds(const T0__& theta, std::ostream* pstream__) {
            return theta / (1 - theta);  
       }
}
```